In [1]:
import subprocess
import shlex
import os
import glob

from subprocess import PIPE
from pathlib import Path

In [2]:
def submit_independent_spatially_explicit(
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    config='()', reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = f"""
    #PBS -lselect={1}:ncpus={1}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent{config},

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [3]:
def submit_independent_spatially_explicit_parallel(
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, partitions=4, memory=16,
    config='()', reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
    
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={partitions}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00
    
    module load mpi
    module load anaconda3/personal

    mpiexec $HOME/necsim-rust/target-mpi/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent{config},

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),
        
        log: "'$TMPDIR'/event_log",

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-mpi/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    
    $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
        logs: [
            "'$(echo $TMPDIR | sed "s:\[\([0-9]\+\)\]:\[\[\]\1\[\]\]:g")'/event_log/*/*"
        ],

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [4]:
for delta_t in [0.5, 0.625, 0.75, 0.875, 1.0, 1.5, 1.75, 2.0, 2.25, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8.0, 10.0, 12.0, 16.0, 20.0]:
    submit_independent_spatially_explicit(
        walltime=6, repeats=50, speciation=0.000001, sample=0.00025,
        config=f"(delta_t:{delta_t})", output=f"delta-t/pbs.{delta_t}",
    )

In [5]:
for step_slice in [1, 2, 3, 4, 6, 8, 10, 12, 16, 20]:
    submit_independent_spatially_explicit(
        walltime=6, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(step_slice:{step_slice})", output=f"step-slice/pbs.{step_slice}",
    )

In [6]:
for event_slice in [100000, 177828, 316228, 1000000, 1778279, 3162278, 10000000]:
    submit_independent_spatially_explicit(
        walltime=8, repeats=25, speciation=0.000001, sample=0.00025, reporters=['Execution()', 'Biodiversity()', 'Counter()'],
        config=f"(parallelism_mode:Monolithic(event_slice:{event_slice}))", output=f"event-slice/pbs.{event_slice}",
    )

In [7]:
for event_slice in [1000, 1778, 3162, 10000, 17783, 31623, 100000, 177828, 316228, 1000000, 1778279, 3162278, 10000000]:
    submit_independent_spatially_explicit(
        walltime=1, repeats=10, speciation=0.0001, sample=0.0000025, reporters=['Execution()', 'Biodiversity()', 'Counter()'],
        config=f"(parallelism_mode:Monolithic(event_slice:{event_slice}))", output=f"event-slice-live/pbs.{event_slice}",
    )

In [8]:
for event_slice in [1000, 1778, 3162, 10000, 17783, 31623, 100000, 177828, 316228, 1000000, 1778279, 3162278, 10000000]:
    submit_independent_spatially_explicit_parallel(
        walltime=1, repeats=10, speciation=0.0001, sample=0.0000025, reporters=['Execution()', 'Biodiversity()', 'Counter()'],
        partitions=1, config=f"(parallelism_mode:Monolithic(event_slice:{event_slice}))", output=f"event-slice-log/pbs.{event_slice}",
    )

In [9]:
for dedup_capacity in [100000, 177828, 316228, 1000000, 1778279, 3162278, 10000000, 17782794, 31622777, 100000000]:
    submit_independent_spatially_explicit(
        walltime=6, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(dedup_cache:Absolute(capacity:{dedup_capacity}))",
        output=f"dedup-capacity/pbs.{dedup_capacity}",
    )

In [10]:
for (partitions, memory) in [(2, 16), (4, 16), (8, 16), (16, 32), (32, 62)]:
    for (speciation, sample) in [
        (0.000001, 0.025), (0.000001, 0.000025),
    ]:
        for communication in [
            0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0
        ]:
            submit_independent_spatially_explicit_parallel(
                walltime=6, repeats=100, memory=memory, speciation=speciation, sample=sample, partitions=partitions,
                config=(
                    f"(delta_t:{2.5}, step_slice:{10}, dedup_cache:Relative(factor:{0.1}), " +
                    f"parallelism_mode:Probabilistic(communication:{communication}))"
                ), output=f"communication/pbs.{partitions}.{speciation}.{sample}.{communication}",
            )
            
for (partitions, memory) in [(2, 16), (4, 16), (8, 16), (16, 32), (32, 62)]:
    for (speciation, sample) in [
        (1.0, 0.025), (0.01, 0.0025), (0.0001, 0.00025),
    ]:
        for communication in [
            0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0
        ]:
            submit_independent_spatially_explicit_parallel(
                walltime=6, repeats=25, memory=memory, speciation=speciation, sample=sample, partitions=partitions,
                config=(
                    f"(delta_t:{2.5}, step_slice:{10}, dedup_cache:Relative(factor:{0.1}), " +
                    f"parallelism_mode:Probabilistic(communication:{communication}))"
                ), output=f"communication/pbs.{partitions}.{speciation}.{sample}.{communication}",
            )